In [ ]:
import pandas as pd

In [ ]:
# list of dictionaries containing county Commissioners/Supervisors
# politically diverse
officials = [
    # Tulsa County, OK
    {"name": "Stan Sallee", "office": "Board of County Commissioners", "county": "Tulsa", "district": "1"},
    {"name": "Lonnie Sims", "office": "Board of County Commissioners", "county": "Tulsa", "district": "2"},
    {"name": "Kelly Dunkerley", "office": "Board of County Commissioners", "county": "Tulsa", "district": "3"},

    # Travis County, TX (includes Austin)
    {"name": "Jeff Travillion", "office": "Commissioner's Court", "county": "Travis", "district": "1"},
    {"name": "Brigid Shea", "office": "Commissioner's Court", "county": "Travis", "district": "2"},
    {"name": "Ann Howard", "office": "Commissioner's Court", "county": "Travis", "district": "3"},
    {"name": "Margaret Gomez", "office": "Commissioner's Court", "county": "Travis", "district": "4"},

    # Campbell County, WY
    # District numbers fabricated
    {"name": "Bob Jordan", "office": "Board of County Commissioners", "county": "Campbell", "district": "1"},
    {"name": "Jerry Means", "office": "Board of County Commissioners", "county": "Campbell", "district": "2"},
    {"name": "Jim Ford", "office": "Board of County Commissioners", "county": "Campbell", "district": "3"},
    {"name": "Scott Clem", "office": "Board of County Commissioners", "county": "Campbell", "district": "4"},
    {"name": "Kelley McCreery", "office": "Board of County Commissioners", "county": "Campbell", "district": "5"},

    # Maricopa County, AZ (includes Phoenix)
    {"name": "Mark Stewart", "office": "Board of Supervisors", "county": "Maricopa", "district": "1"},
    {"name": "Thomas Galvin", "office": "Board of Supervisors", "county": "Maricopa", "district": "2"},
    {"name": "Kate Brophy McGee", "office": "Board of Supervisors", "county": "Maricopa", "district": "3"},
    {"name": "Debbie Lesko", "office": "Board of Supervisors", "county": "Maricopa", "district": "4"},
    {"name": "Steve Gallardo", "office": "Board of Supervisors", "county": "Maricopa", "district": "5"},
]

In [ ]:
import random


# Create realistic messy variations of an official's record
def create_variations(official: dict) -> list:
    name = official['name']
    office = official['office']
    county = official['county']
    district = official.get('district', '')

    location = f"{county} County"

    # Parse name
    parts = name.split()
    first = parts[0]
    last = parts[-1]
    middle = parts[1] if len(parts) > 2 else ""

    # Remove quotes from nicknames if present (like "Bobby")
    first_clean = first.replace('"', '').replace("'", '')

    variations = []

    # 1. Clean original record
    variations.append({
        'record_id': f"{county}_{last}_{first}_clean",
        'name_raw': name,
        'title_raw': f"{office} - District {district}" if district else office,
        'location': location,
        'source': 'county website',
        'true_id': name  # Ground truth for evaluation
    })

    # 2. News article style (casual title + last name)
    title_short = "Commissioner" if "Commissioner" in office else "Supervisor"
    variations.append({
        'record_id': f"{county}_{last}_{first}_news",
        'name_raw': f"{title_short} {last}",
        'title_raw': "",
        'location': location,
        'source': 'news_article',
        'true_id': name
    })

    # 3. Meeting minutes (abbreviated, formal)
    initial = first_clean[0]
    variations.append({
        'record_id': f"{county}_{last}_{first}_minutes",
        'name_raw': f"{last}, {initial}.",
        'title_raw': f"District {district}" if district else "Member",
        'location': location,
        'source': 'meeting_minutes',
        'true_id': name
    })

    # 4. Formal full name with title
    variations.append({
        'record_id': f"{county}_{last}_{first}_formal",
        'name_raw': name,
        'title_raw': f"{county} County {title_short}, Dist. {district}" if district else f"{county} County {title_short}",
        'location': location,
        'source': 'public_records',
        'true_id': name
    })

    # 5. Email/contact list style (last, first)
    variations.append({
        'record_id': f"{county}_{last}_{first}_email",
        'name_raw': f"{last}, {first_clean}",
        'title_raw': f"D{district}" if district else "",
        'location': location,
        'source': 'email_list',
        'true_id': name
    })

    return variations

In [ ]:
# Generate all variations
all_records = []
for official in officials:
  variations = create_variations(official)
  all_records.extend(variations)

In [ ]:
# Build df with messy data
df_messy = pd.DataFrame(all_records)

df_messy.head(20)

In [ ]:
# Load data to BigQuery (using default GCP authentication)
from google.cloud import bigquery
import pandas as pd

PROJECT_ID = "clean-energy-projects"

# Upload DataFrame to BigQuery as new table
df_messy.to_gbq(
    destination_table='county_officials_demo.officials',
    project_id=PROJECT_ID,
    if_exists='replace'  # Replace if table exists
)